In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 89.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 103.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 25.0 MB/s eta 0:00:00


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
import tensorflow as tf
from sklearn.metrics import confusion_matrix, classification_report
from keras.callbacks import EarlyStopping
from transformers import TFBertForSequenceClassification, BertTokenizer

In order for our deep learning models to run as fast as possible, we'll need access to a GPU.

In Google Colab, you can set this up by going to Runtime -> Change runtime type -> Hardware accelerator -> GPU.

After selecting GPU, you may have to restart the runtime.

In [ ]:
# Check for GPU
!nvidia-smi -L

GPU 0: Tesla T4 (UUID: GPU-ed9dbfe3-4978-7a74-6394-872e30b12e90)


In [ ]:
# Read in CSV Data for Twitter Sentiment Analysis
df = pd.read_csv("twitter_sentiment_data.csv")

# Check distribution of data
sentiment_counts = df['sentiment'].value_counts()

# Get sentences and labels as dataframes
sentences = df["message"].astype(str).to_numpy()
labels = df["sentiment"].to_numpy()

# Remove -1
labels[labels == -1] = 3

# One-hot encode the labels
num_classes = 4
labels = tf.keras.utils.to_categorical(labels, num_classes)

# Split into train and test
train_sentences, test_sentences, train_labels, test_labels = train_test_split(sentences, labels, test_size=0.2, random_state=1)

# Calculate class weights
class_weights = {}
for i in range(num_classes):
    class_count = np.sum(labels[:, i])
    class_weight = len(labels) / (num_classes * class_count)
    class_weights[i] = class_weight

# Load the BERT tokenizer and model
model_name = "bert-base-uncased"
tokenizer = BertTokenizer.from_pretrained(model_name)
model = TFBertForSequenceClassification.from_pretrained(model_name, num_labels=num_classes)

# Tokenize train and test sentences
train_encodings = tokenizer(list(train_sentences), truncation=True, padding=True, max_length=128)
test_encodings = tokenizer(list(test_sentences), truncation=True, padding=True, max_length=128)

# Convert tokenized data to TensorFlow datasets
train_dataset = tf.data.Dataset.from_tensor_slices((dict(train_encodings), train_labels)).batch(8)
test_dataset = tf.data.Dataset.from_tensor_slices((dict(test_encodings), test_labels)).batch(8)

# Compile the model
model.compile(optimizer=tf.keras.optimizers.Adam(),
              loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
              metrics=["accuracy"])

# Create EarlyStopping instance
early_stopping = EarlyStopping(monitor="val_loss", patience=3, restore_best_weights=True)

# Fit the model
history = model.fit(train_dataset,
                    epochs=10,
                    validation_data=test_dataset,
                    class_weight=class_weights,
                    callbacks=[early_stopping])

# Evaluate the model on the test set
results = model.evaluate(test_dataset)
print(f"Loss: {results[0]}, Accuracy: {results[1]}")

# Make predictions
predictions = model.predict(test_dataset)

# Convert predictions to labels
pred_labels = np.argmax(predictions.logits, axis=1)

# Print the confusion matrix
cm = confusion_matrix(np.argmax(test_labels, axis=1), pred_labels)
print(cm)

# Print the classification report
cr = classification_report(np.argmax(test_labels, axis=1), pred_labels)
print(cr)

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/10
4395/4395 [==============================] - 1172s 255ms/step - loss: 1.4909 - accuracy: 0.2380 - val_loss: 1.4047 - val_accuracy: 0.2123
Epoch 2/10
2119/4395 [=============>................] - ETA: 8:45 - loss: 1.4115 - accuracy: 0.2373

KeyboardInterrupt: ignored